In [ ]:
#importing the libararies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score

In [ ]:
df= pd.read_csv('sentimentdataset.csv')

In [ ]:
df.head(5)

,ID,Text,Sentiment (Label),Timestamp,User,Source,Topic,Retweets,Likes,Country,Year,Month,Day,Hour
0,0,Enjoying a beautiful day at the park! ...,Positive,1/15/2023 12:30,User123,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,1,Traffic was terrible this morning. ...,Negative,1/15/2023 8:45,CommuterX,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,2,Just finished an amazing workout! 💪 ...,Positive,1/15/2023 15:45,FitnessFan,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,3,Excited about the upcoming weekend getaway! ...,Positive,1/15/2023 18:20,AdventureX,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,4,Trying out a new recipe for dinner tonight. ...,Neutral,1/15/2023 19:55,ChefCook,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19


In [ ]:
df['Sentiment (Label)'].nunique()

279

In [ ]:
df['Source'].nunique()

4

In [ ]:
df.drop(columns=['ID','User'],inplace= True)

In [ ]:
df.head(5)

,Text,Sentiment (Label),Timestamp,Source,Topic,Retweets,Likes,Country,Year,Month,Day,Hour
0,Enjoying a beautiful day at the park! ...,Positive,1/15/2023 12:30,Twitter,#Nature #Park,15,30,USA,2023,1,15,12
1,Traffic was terrible this morning. ...,Negative,1/15/2023 8:45,Twitter,#Traffic #Morning,5,10,Canada,2023,1,15,8
2,Just finished an amazing workout! 💪 ...,Positive,1/15/2023 15:45,Instagram,#Fitness #Workout,20,40,USA,2023,1,15,15
3,Excited about the upcoming weekend getaway! ...,Positive,1/15/2023 18:20,Facebook,#Travel #Adventure,8,15,UK,2023,1,15,18
4,Trying out a new recipe for dinner tonight. ...,Neutral,1/15/2023 19:55,Instagram,#Cooking #Food,12,25,Australia,2023,1,15,19


In [ ]:
!pip install vaderSentiment

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 987.2 kB/s eta 0:00:00


## **Sentiment Intensity Analyzer**

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
df['Sentiment (Label)']=df['Sentiment (Label)'].str.lower().str.strip()
analyzer = SentimentIntensityAnalyzer()

# Define a function to categorize sentiment words using VADER
def categorize_sentiment(word):
    score = analyzer.polarity_scores(word)['compound']
    if score >= 0.05:
        return 'positive'
    elif score <= -0.05:
        return 'negative'
    else:
        return 'neutral'

df['sentiment_category (y)'] = df['Sentiment (Label)'].apply(categorize_sentiment)

for words in df[['Sentiment (Label)','sentiment_category (y)']].values:
    print(words)

['positive' 'positive']
['negative' 'negative']
['positive' 'positive']
['positive' 'positive']
['neutral' 'neutral']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['negative' 'negative']
['neutral' 'neutral']
['positive' 'positive']
['negative' 'negative']
['positive' 'positive']
['positive' 'positive']
['neutral' 'neutral']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['neutral' 'neutral']
['negative' 'negative']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']
['positive' 'positive']


In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['sentiment_category (y)'] = label_encoder.fit_transform(df['sentiment_category (y)'])

# **Count Vectorizer**

After finding that the unique values of object columns are more than 10, a decision was made to use count Vectorizer instead of label encoding.

In [ ]:
import nltk
nltk.download ('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
port_stem=PorterStemmer()

In [ ]:
print(df['Text'])

0       Enjoying a beautiful day at the park!        ...
1       Traffic was terrible this morning.           ...
2       Just finished an amazing workout! 💪          ...
3       Excited about the upcoming weekend getaway!  ...
4       Trying out a new recipe for dinner tonight.  ...
                             ...                        
727    Collaborating on a science project that receiv...
728    Attending a surprise birthday party organized ...
729    Successfully fundraising for a school charity ...
730    Participating in a multicultural festival, cel...
731    Organizing a virtual talent show during challe...
Name: Text, Length: 732, dtype: object


In [ ]:
def preprocess_text(text):
    # Tokenize the text
    tokens = nltk.word_tokenize(text)
    # Remove non-alphabetic words and convert to lowercase
    tokens = [word.lower() for word in tokens if word.isalpha()]
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Join tokens back into a string
    return ' '.join(tokens)

In [ ]:
df['Processed_Text'] = df['Text'].apply(preprocess_text)

In [ ]:
print(df['Processed_Text'])

0                            enjoying beautiful day park
1                               traffic terrible morning
2                               finished amazing workout
3                       excited upcoming weekend getaway
4                       trying new recipe dinner tonight
                             ...                        
727    collaborating science project received recogni...
728    attending surprise birthday party organized fr...
729    successfully fundraising school charity initia...
730    participating multicultural festival celebrati...
731    organizing virtual talent show challenging tim...
Name: Processed_Text, Length: 732, dtype: object


In [ ]:
def stem_text(text):
    tokens = word_tokenize(text)  # Tokenize the text
    stemmed_tokens = [port_stem.stem(token) for token in tokens]
    #stemmed_tokenst=re.sub('[^a-zA-z]','',stemmed_tokens)# Stem each token
    return ' '.join(stemmed_tokens)  # Join stemmed tokens back into a string

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
 df['Stemmed_Text'] = df['Processed_Text'].apply(preprocess_text)

In [ ]:
print(df['Stemmed_Text'])

0                            enjoying beautiful day park
1                               traffic terrible morning
2                               finished amazing workout
3                       excited upcoming weekend getaway
4                       trying new recipe dinner tonight
                             ...                        
727    collaborating science project received recogni...
728    attending surprise birthday party organized fr...
729    successfully fundraising school charity initia...
730    participating multicultural festival celebrati...
731    organizing virtual talent show challenging tim...
Name: Stemmed_Text, Length: 732, dtype: object


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
tfidf_features = vectorizer.fit_transform(df['Stemmed_Text'])
tfidf_featuresss = vectorizer.fit_transform(df['Topic'])

In [ ]:
#connecting the preprocessed columns
X = pd.concat([pd.DataFrame(tfidf_features.toarray())
             ,pd.DataFrame(tfidf_featuresss.toarray())], axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,df['sentiment_category (y)'], test_size=0.2, random_state=5)

In [ ]:
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [ ]:
from sklearn.naive_bayes import MultinomialNB

model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
# Perform cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(model, X, df['sentiment_category (y)'], cv=kfold)

In [ ]:
# Print the cross-validation scores
print("Cross-validation scores:", scores)

Cross-validation scores: [0.80272109 0.86394558 0.86986301 0.83561644 0.83561644]


In [ ]:
# Print the mean and standard deviation of the cross-validation scores
print("Mean accuracy:", scores.mean())
print("Standard deviation:", scores.std())

Mean accuracy: 0.8415525114155251
Standard deviation: 0.024005363227131767


In [ ]:
predictions = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Accuracy:", accuracy)

Accuracy: 0.7891156462585034


# **Train the Logistic Regression**

In [ ]:
classifier = LogisticRegression(random_state = 0)

In [ ]:
scores_log = cross_val_score(classifier, X, df['sentiment_category (y)'], cv=kfold)

In [ ]:
# Print the cross-validation scores
print("Cross-validation scores:", scores_log)

Cross-validation scores: [0.68707483 0.82312925 0.81506849 0.69863014 0.78082192]


In [ ]:
print("Mean accuracy:", scores_log.mean())
print("Standard deviation:", scores_log.std())

Mean accuracy: 0.7609449259155717
Standard deviation: 0.057500034278647784


In [ ]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
scores_log = cross_val_score(model, X, df['sentiment_category (y)'], cv=kfold)

In [ ]:
# Print the mean and standard deviation of the cross-validation scores
print("Mean accuracy:", scores_log.mean())
print("Standard deviation:", scores_log.std())

In [ ]:
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test,predictions)
cm

In [ ]:
cr=classification_report(y_test,predictions)
print(cr)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


In [ ]:
# Initialize the SVM classifier
svm_classifier = SVC(kernel='linear')  # You can choose different kernels such as 'linear', 'rbf', 'poly', etc.

# Train the SVM classifier
svm_classifier.fit(X_train, y_train)

SVC(kernel='linear')

In [ ]:
svm_scores = cross_val_score(svm_classifier, X, df['sentiment_category (y)'], cv=kfold)

In [ ]:
# Print the cross-validation scores
print("Cross-validation scores:", svm_scores)

Cross-validation scores: [0.80952381 0.85714286 0.84931507 0.82191781 0.84931507]


In [ ]:
# Print the mean and standard deviation of the cross-validation scores
print("Mean accuracy:", svm_scores.mean())
print("Standard deviation:", svm_scores.std())

Mean accuracy: 0.8374429223744292
Standard deviation: 0.018387436845346744


In [ ]:
# Predict on the test set
y_pred_svm = svm_classifier.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred_svm)
print("Accuracy:", accuracy)

Accuracy: 0.8095238095238095


In [ ]:
x= (input())
y=(input())

roro
lol


In [ ]:
pross_x = preprocess_text(x)
pross_x = stem_text (pross_x )
r=vectorizer.transform(pross_x)

ValueError: Iterable over raw text documents expected, string object received.

In [ ]:
pross_y = preprocess_text(y)
pross_y = stem_text (pross_y)
pross_y=vectorizer.fit_transform(pross_y)

In [ ]:
model.predict(pross_x)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  warnings.warn(


ValueError: Expected 2D array, got scalar array instead:
array=roaa.
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
pipeline = Pipeline([
    ('vect', CountVectorizer(max_features=4)),  # You can set max_features here if needed
    ('clf', RandomForestClassifier())  # You can specify model parameters here
])
